![Alt text](http://www.ucm.es/logo/ucm.png "a title")

<div align="center"> 
<font size=5> Máster en Nuevas Tecnologías Electrónicas y Fotónicas </font>
</div>
    
<div align="center"> 
<font size=4> Óptica Digital, curso 2021-2022 </font>
</div>

    
<div align="center"> 
<font size=4> Ejercicio A - Introducción a Py_pol </font>
</div>

- **Fecha**: *
        
- **Alumno**: *  


# Introducción

Este es un ejercicio sencillo para empezar a utilizar el módulo *py_pol*.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from py_pol.jones_matrix import Jones_matrix
from py_pol.jones_vector import Jones_vector

# 1. Generador de estados

Vamos a modelar un generador de estados compuesto por los mismos elementos que vemos en clase y en el laboratorio: una fuente de luz circularmente polarizada y un polarizador y una lámina de cuarto de onda rotatorios. Utilizaremos elementos ideales por simplicidad.

El ejercicio consiste en una demostración ilustrativa de las ecuaciones que relacionan los ángulos de los elementos del PSG con el acimut y el ángulo de elipticidad del estado generado. Para ello tendréis que realizar los siguientes pasos.

1. Crear un elemento *Jones_matrix* que represente al polarizador y otro al retardador. Estos elementos deben tener todas las posibles combinaciones de ángulos de giro (entre 0º y 180º) con una resolución mínima de 1º. Para esto será de gran ayuda la función *meshgrid* de *numpy*.
2. Crear la fuente circularmente polarizada como un *Jones_vector*. Multiplicar adecuadamente los elementos para obtener el estado a la salida del PSG.
3. Representar gráficamente el aimut y el ángulo de elipticidad del estado final. Para ello serán de gran ayuda los métodos *azimuth* y *ellipticity* de la subclase *Jones_vector.parameters*.
4. Representar gráficamente el resultado de aplicar las ecuaciones dadas en clase que calculan el acimut y el ángulo de elipticidad del estado final a partir de los ángulos de los elementos del PSG.
5. Analizar las diferencias.

# 2. Analizador de estados

Ahora vamos a modelar el analizador de estados, de nuevo con una lámina de cuarto de onda y un polarizador rotatorios. De nuevo, serán elementos ideales.

El ejercicio consistirá en una demostración visual de que el estado analizado es aquel que tiene mayor transmisión (1 porque emplearemos elementos ideales) y su estado ortogonal la mínima (0 en el caso de elementos ideales). Para ello debéis realizar los siguientes pasos.

1. Crear dos objetos *Jones_matrix* que representen al retardador y polarizador del PSA. Escoger sus ángulos para que el estado analizado tenga un acimut y un ángulo de elipticidad conocidos pero generados aleatoriamente (por ejemplo, empleando la función *numpy.random.randint*). 
2. Crear un objeto *Jones_vector* que va a ser el estado incidente. Este objeto tiene que contener todas las combinaciones posibles de acimut y ángulo de elipticidad (entre 0º y 180º y -45º y 45º) con una resolución mínima de 1º.
3. Multiplicar los elementos de manera que se pueda calcular el estado final.
4. Calcular y representar la transmisión de intensidad del PSA.
5. Identificar los puntos de transmisión 0 y 1. Pueden ser de utilidad las funciones *argmin*, *argmax* y *unravel_index* de *numpy*.